<h1>Importing</h1>

In [ ]:
import pandas as pd 
import os
import shutil
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
from torchvision.transforms import ToTensor
import PIL
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from tqdm import tqdm
import tensorflow as tf

In [ ]:
path_datasset = "train.csv"
sample_dataset = "sample_submission.csv"
test_dataset = "test-2.csv"
df = pd.read_csv(path_datasset)
df.head()

In [ ]:
df_sample = pd.read_csv(sample_dataset)
df_test = pd.read_csv(test_dataset)

df_test['has_under_extrusion'] = df_sample['has_under_extrusion'].values.astype(int)
df_test

In [ ]:
df_test.to_csv('./test-2.csv', index=False)
df_test = pd.read_csv('./test-2.csv')

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform

    def __len__(self):
        return len(self.df)
    def __getitem__(self, index):
        image_path = self.df.loc[index]['img_path']
        label = self.df.iloc[index][-1].astype(float)
        image = PIL.Image.open(os.path.join(self.images_folder, image_path))
        if self.transform is not None:
            image = self.transform(image)
        label = torch.tensor(label, dtype=torch.float32).unsqueeze(0)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.AutoAugment(),
    transforms.ToTensor(),
    transforms.Resize(size=(224, 224), antialias=True),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = CustomDataset(csv_path = path_datasset, images_folder = "./images", transform = transform)
train_dataset,_=torch.utils.data.random_split(dataset, [0.8,0.2],)
test_dataset = CustomDataset(csv_path = test_dataset, images_folder = "./images", transform = transform)


In [ ]:
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)


In [ ]:
test_dataset = CustomDataset(csv_path = test_dataset, images_folder = "./images", transform = transform)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.efficientnet_b0(pretrained=True)
model = torch.load('model-ENB0.pth')

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)
size = len(train_dataset)
epochs = 5

In [ ]:
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0
    
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward Pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        
        # Backward Pass and Optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        predicted = (outputs > 0).float()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)
    
    train_loss /= len(train_loader)
    train_accuracy = 100 * train_correct / train_total

print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
print("Finished Training")

In [ ]:
# # export model weights
# torch.save(model.state_dict(), 'model.pth')

In [ ]:
model.eval()  # Set the model to evaluation mode

test_correct = 0
test_total = 0
test_loss = 0
with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()  # binary classification
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)
    
    test_accuracy = 100 * test_correct / test_total

print(f'Test Accuracy: {test_accuracy}')